## Amazon User-Profile Crawling Data Cleaning
Step3 Cleaning Data
 
Merge Both Setp1 and Step2 Data and `Cleaning for further Step Analyzing`, including Data type transforming, Punctuation cleaning, Time thershold etc. 

#### Preparing Data
- Step 1 Folder: Including Data Crawling From Step 1
- Step 2 Folder: Including Data Crawling From Step 2 



In [ ]:
import pandas as pd
from glob import glob
import re


In [ ]:
step1 = glob('Step1/*.csv')
df1 = pd.DataFrame()
for file in step1:
    tmp = pd.read_csv(file)
    tmp['Belong_Model'] = file.replace('Step1/AMZ_PH_STEP1_', '')
    df1 = df1.append(tmp, ignore_index=True)
df1 = df1.drop(columns=['Unnamed: 0'])
for i in range(len(df1)):
    df1['Asin'][i] = re.sub(r'[^a-zA-Z0-9]', "", df1['Asin'][i])
df1.index = range(len(df1.index))
# df1.to_csv('Step1_df1.csv')
print(df1.info(), df1.head())


In [ ]:
step2 = glob('Step2/*.csv')
df2 = pd.concat((pd.read_csv(file) for file in step2), ignore_index=True)

df2 = df2.drop(columns=['Unnamed: 0'])
for i in range(len(df2)):
    for cl in ['Category', 'Sub_category', 'Brand']:
        df2[cl][i] = (re.sub(r'[^a-zA-Z0-9]', "", df2[cl][i])).replace(' ', '')

    for cla in ['Asin']:
        df2[cla][i] = re.sub(r'[^a-zA-Z0-9]', "", df2[cla][i])
print(df2.info(), df2.head())
# df2.to_csv('Step2_df2.csv')


In [ ]:
format = pd.merge(df1, df2, on='Asin')
df3 = format.drop_duplicates(subset=['review_content'])
df3.index = range(len(df3.index))


In [ ]:
# 調整時間格式
print(
    '時間處理前:',
    type(df3['review_date_place'][0]),
    df3['review_date_place'][0],
)

for t in range(len(df3['review_date_place'])):
    time_raw = str(df3['review_date_place'][t]).split('·')[1]
    # 若是中文時間格式則取消註解

    time_raw = time_raw.replace('年', '-')
    time_raw = time_raw.replace('月', '-')
    time_raw = time_raw.replace('日', '')
    df3['review_date_place'][t] = time_raw

df3['review_date_place'] = pd.to_datetime(df3['review_date_place'])

print(
    '時間處理後:',
    type(df3['review_date_place'][0]),
    df3['review_date_place'][0],
)


In [ ]:
start_time = '2018-01-01'
end_time = '2022-12-31'

df3 = df3.loc[
    (df3['review_date_place'] >= start_time)  # 時間起始點
    & (df3['review_date_place'] < end_time)
]  # 時間終點
print(df3.info(), df3.head())


In [ ]:
def replace_del(text, del_list):
    for i in del_list:
        text = text.replace(i, "")
    return text


df3['Price'] = [replace_del(p, ['\'', '[', ']']) for p in df3['Price']]
df3['Brand'] = [replace_del(p, ['\'', '[', ']']) for p in df3['Brand']]
df3['review_rating'] = [
    replace_del(star, [' out of five stars']) for star in df3['review_rating']
]

df3.head()
df3.to_csv('M_WTG_AMZ_Profile_All_Data.csv', index=False)
